<a href="https://colab.research.google.com/github/shreevigneshs/UCSC-243/blob/main/assignments/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import List

import torch
import torchtext
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from collections import Counter

from tqdm import tqdm

In [2]:
# !python -m spacy download en

In [3]:
# !pip install -U nltk

In [4]:
# from nltk.tokenize import tokenizer

In [5]:
df = pd.read_csv("/content/drive/MyDrive/NLP243/assignments/1/hw1_train-1.csv")

In [6]:
print(df)

        ID                                         UTTERANCES  \
0        0               who plays luke on star wars new hope   
1        1                     show credits for the godfather   
2        2             who was the main actor in the exorcist   
3        3  find the female actress from the movie she's t...   
4        4                    who played dory on finding nemo   
...    ...                                                ...   
2307  2307               what was the revenue for toy story 3   
2308  2308                                dark knight revenue   
2309  2309               how much did the dark night generate   
2310  2310                  can i see the lion king's revenue   
2311  2311         can i see what the lion king's revenue was   

                                     CORE RELATIONS  
0     movie.starring.actor movie.starring.character  
1                              movie.starring.actor  
2                              movie.starring.actor  
3  

In [7]:
df.columns = ["id", "text", "labels"]

In [8]:
df.labels = df.labels.replace(np.nan, "none", regex=True)

In [9]:
labels = list(set([_split_label for label in df.labels.unique().flatten() for _split_label in label.split()]))

In [10]:
print(labels)
print(len(labels))

['movie.estimated_budget', 'movie.production_companies', 'gr.amount', 'movie.subjects', 'movie.gross_revenue', 'movie.language', 'movie.locations', 'movie.starring.actor', 'movie.starring.character', 'movie.initial_release_date', 'person.date_of_birth', 'movie.genre', 'none', 'actor.gender', 'movie.produced_by', 'movie.country', 'movie.rating', 'movie.directed_by', 'movie.music']
19


In [11]:
tokens = list(set([str.lower(_word) for text in df.text for _word in text.split()]))

In [12]:
len(tokens)

1192

In [13]:
tokens = [_token[:_token.index("'")] if "'" in _token else _token for _token in tokens]

In [14]:
import string
table = str.maketrans('', '', string.punctuation)
stripped = [token.translate(table) for token in tokens]
# remove remaining tokens that are not alphabetic
tokens = [token for token in stripped if token.isalpha()]

In [15]:
stop_words = list(set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']))

In [16]:
tokens = [token for token in tokens if not token in stop_words]


In [17]:
len(tokens)

1096

In [18]:
token_2 = [_token for _token in tokens if len(_token) == 2]

In [19]:
token_1 = [_token for _token in tokens if len(_token) == 1]

In [20]:
print(len(token_1))
print(token_1)

4
['b', 'r', 'g', 'e']


In [21]:
print(token_2)

['yr', 'ok', 'iv', 'de', 'et', 'ju', 'uk', 'le', 'pi', 'pg', 'wb', 'ms', 'la', 'ed', 'th', 'go', 'nc', 'et', 'fi', 'oz', 'jj', 'us']


In [22]:
all_tokens = Counter(tokens)

In [23]:
len(all_tokens)

1061

In [24]:
en_tokenizer = get_tokenizer('spacy', language='en')

def build_vocab(text, tokenizer):
  counter = Counter()
  counter.update(tokenizer(text))
  return Vocab(counter)

en_vocab = build_vocab(" ".join([text for text in df.text]), en_tokenizer)

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:106: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead'


In [25]:
len(en_vocab)

1155

In [26]:
t = torch.tensor([6,6,7,8,6,1,7], dtype = torch.int64)
one_hot_vector = F.one_hot(x = t, num_classes=9)
print(one_hot_vector)

tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0]])


In [27]:
b = torch.zeros(7, 9, dtype=torch.int64)

In [28]:
c = torch.cat((one_hot_vector, b), 0)
print(c)

tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [60]:

from numpy import argmax
# define the text string
data = 'hello'
# define universe of possible input values
alphabet = 'abcdefghijklmnopqrstuvwxyz '
# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# integer encode input data
integer_encoded = [char_to_int[char] for char in data]
print(integer_encoded)
# convert tensors into one-hot encoding 
torch.nn.functional.one_hot(torch.tensor(integer_encoded))

[7, 4, 11, 11, 14]


tensor([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [46]:
from torch.functional import norm
class MovieDataset(Dataset):
    def __init__(
        self,
        path: str,  # Path to the training data csv
        vocab_size: int = 100  # How many tokens to include in the vocabulary. Feel free to adjust this!
    ):
        self.input_text_len = 50
        self.stop_words = list(set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']))
        
        # Read the csv using pandas read_csv function.
        self.data = pd.read_csv(path, index_col=0)

        # the given column names are long, so I often rename them for simplicity.
        self.data.columns = ["text", "labels"]

        # There's a problem with this data...some of the rows have a label called 'none', and others
        # are just empty. These are both referring to the same condition, so lets replace the 'none'
        # labels with empty strings to make it easier. Otherwise, you might predict both 'none' and
        # another label, which doesn't make any sense.
        self.data["labels"] = self.data.labels.replace(np.nan, "none", regex=True)

        # For one-hot encoding, we need a list of all unique labels in the dataset and a map between
        # labels and a unique ID.
        
        # TODO create self.labels: a list of every possible label in the dataset
        # e.g., ['movie.starring.actor', 'movie.gross_revenue', ...]
        # ======================================================================
        self.labels = list(set([_split_label for label in df.labels.unique().flatten() for _split_label in label.split()]))
        self.n_labels = len(self.labels)
        
        # TODO create self.label2id: a dictionary which maps the labels from self.labels (above) to a unique integer
        # ======================================================================
        self.label2id = {}
        _cnt = 0
        for _label in self.labels:
          self.label2id[_label] = _cnt
          _cnt += 1
        
        # Similarly, we often need to make a token vocabulary for encoding the input text. Note that
        # this isn't necessary for ALL representations, but for some, you will find it useful. 
        # However, we are only creating the vocabulary from the training data. What happens if 
        # the test data has a token we haven't seen before? 
        # To combat this, we default to a particular token, usually something like <unk> ('unknown').
        
        # In the future, you will see datasets with hundreds of thousands of unique tokens. Normally,
        # we only take the N most common tokens and replace everything else with <unk>. 
        # Otherwise, our models would be huge! For this dataset, it's not a problem, but you should know
        # how to do it. 
        
        # TODO create self.vocab: a dictionary which contains the `vocab_size` most common tokens in the text.
        # Here's a hint - check out the `Counter` class from python's `collections` library.
        # ======================================================================
        
        self.vocab = {}

        # also, don't forget to include <unk> (unknown)
        # TODO assign <unk> a unique ID. 
        # ======================================================================

        _cnt = 0
        self.vocab['<unk>'] = 0
        _cnt += 1

        all_vocab = Counter(list(set([_word for text in self.data.text for _word in self.tokenize(text=text)])))

        most_common_vocab = all_vocab.most_common(vocab_size-1)

        for _x in most_common_vocab:
          self.vocab[_x[0]] = _cnt
          _cnt += 1

        self.vocab_size = _cnt
        # self.vocab_size = vocab_size + 1 # plus 1 because <unk>
    
    def one_hot_encode_labels(self, label: str):
        # For multi-label classification, we're going to one-hot encode our labels.
        # This means that instead of having out data be pairs like:
        #   {'input': ..., 'output': 2}
        # We instead might have multiple correct classes, so we do something like
        #   {'input': ..., 'output': [0, 0, 1, 0, ...]}
        # where the output is a list with one element per possible label. Then, a 1 in position N means
        # the label N is a correct answer.
        
        # We need to create such a list from the input to this function, `labels`, which is a list
        # of labels that appear in a particular example. It might be, for example, 
        #   ['movie.starring.actor', 'movie.release_date']
        # Good thing we have self.label2id! That should help us figure out which 
        # index corresponds to which label, so we can write our own function. 
        # Although...this is a very common thing to do in NLP,
        # I'm sure it's available in a library somewhere (hint: sklearn). 
        
        # TODO create encoded: a vector (np.array) which is a one-hot encoded 
        # representation of the input,`labels`. 
        # ======================================================================

        # encoded = labels
        encoded = np.zeros(self.n_labels, dtype=int)
        normalized_label = list(set([_split_label for _split_label in label.split()]))
        for _label in normalized_label:
          encoded[self.label2id[_label]] = 1.
        return encoded
    
    def tokenize(self, text: str):
        # Luckily, this dataset is already tokenized; that is, each token is separated by a single
        # spce. Normally, text has punctuation, hyphenated words, paragraph breaks, etc.. which
        # makes tokenization a more complicated problem. For now, just .split() is good enough.
        
        tokens = list(set([str.lower(_word) for _word in text.split()]))
        tokens = [_token[:_token.index("'")] if "'" in _token else _token for _token in tokens]

        table = str.maketrans('', '', string.punctuation)
        stripped = [token.translate(table) for token in tokens]
        tokens = [token for token in tokens if not token in stop_words]
        tokens = [token for token in stripped if token.isalpha()]
        return tokens
    
    def encode_tokens(self, tokens: List[str]):
        # Think about how you want to encode your tokens. One-hot encoding? Something else 
        # you've learned in 220 or 243?
        # Whatever you decide, it's convenient if you are able to feed the output of this
        # function directly into your model, like this:
        #   >>> model(encode_tokens(['this', 'is', 'a', 'sentence']))
        
        # Note: that's only a suggestion, you don't have to feed this directly into the model.
        # Feel free to set up your data/model pipeline as you see fit. 
        
        # TODO create encoded: an encoded representation of `tokens`.
        # ======================================================================

        # encoded = tokens
        token_ids = [self.vocab[_token] if _token in self.vocab else self.vocab["<unk>"] for _token in tokens]
        if self.input_text_len < len(token_ids):
          token_ids = token_ids[:self.input_text_len]
        t = torch.tensor(token_ids, dtype = torch.int64)
        one_hot_vector = F.one_hot(x = t, num_classes=self.vocab_size)
        if self.input_text_len > len(token_ids):
          padding = torch.zeros(self.input_text_len - len(token_ids), self.vocab_size, dtype=torch.int64)
          encoded = torch.cat((one_hot_vector, padding), 0)
        else:
          encoded = one_hot_vector
        return encoded

    def __len__(self):
        # PyTorch expects every Dataset class to implement the __len__ function. 
        # Most of the time, it's very simple like this.
        return len(self.data)

    def __getitem__(self, n: int):
        # TODO get the nth item of your data, and process it so it's ready to used in your model
        # and for training.
        # ======================================================================
        
        # Make sure the output of this function is either an np.array, a
        # torch.Tensor, or a tuple of several of these. That way, pytorch
        # can combine them into batches properly using its default collate_fn in the DataLoader.
        # If you're using nn.Embedding, you will have to deal with padding,
        # but I'll leave that for you :)

        text = self.data.text[n]
        labels = self.data.labels[n]
        return self.encode_tokens(text).to(torch.float), self.one_hot_encode_labels(labels).astype(float)



In [47]:
# Instantiate the data; you'll need to upload this file, or download the notebook and run it locally if you want this to work. 
dataset = MovieDataset("/content/drive/MyDrive/NLP243/assignments/1/hw1_train-1.csv")

# A small batch size of 2 makes it easier to debug for printing. 
data_loader = DataLoader(dataset, batch_size=2)

In [68]:
# Zipping the dataloader with range(N) lets us only print the first N batches
for _, batch in zip(range(5), data_loader):
    # Do something here; maybe print the batch to see if it looks right to you?
    print(batch[0].shape, batch[1].shape)

torch.Size([2, 50, 100]) torch.Size([2, 19])
torch.Size([2, 50, 100]) torch.Size([2, 19])
torch.Size([2, 50, 100]) torch.Size([2, 19])
torch.Size([2, 50, 100]) torch.Size([2, 19])
torch.Size([2, 50, 100]) torch.Size([2, 19])


In [ ]:
class MLP(nn.Module):
def __init__(self, vocab_size=100, input_size=50, hidden_size=10, output_size=19, dropout=False, dropout_p=0.1):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(vocab_sizeembedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [61]:
class MultiLayerPerceptron(nn.Module):

    def __init__(self, input_size=50, hidden_size=100, output_size=19, dropout=False, dropout_p=0.1):
        super(MultiLayerPerceptron, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size, bias=True)
        self.hidden_layer = nn.Linear(hidden_size, output_size, bias=True)

        self.add_dropout = dropout
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        h = F.relu(self.input_layer(x).view(1, -1))
        if self.add_dropout:
            logits = self.hidden_layer(self.dropout(h))
        else:
            logits = self.hidden_layer(h)

        return logits

In [62]:
print(MultiLayerPerceptron())

MultiLayerPerceptron(
  (input_layer): Linear(in_features=50, out_features=100, bias=True)
  (hidden_layer): Linear(in_features=100, out_features=19, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [64]:
class MultiClassTrainer(object):
    """
    Trainer for training a multi-class classification model
    """

    def __init__(self, model, optimizer, loss_fn, device="cpu", log_every_n=None):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device
        self.loss_fn = loss_fn
        
        self.log_every_n = log_every_n if log_every_n else 0


    def _print_summary(self):
        print(self.model)
        print(self.optimizer)
        print(self.loss_fn)

    def train(self, loader):
        """
        Run a single epoch of training
        """

        self.model.train() # Run model in training mode

        loss_history = []
        running_loss = 0.
        running_loss_history = []

        for i, batch in tqdm(enumerate(loader)):
            batch_size = batch[0].shape[0]
            self.optimizer.zero_grad() # Always set gradient to 0 before computing it

            logits = self.model(batch[0].to(self.device)) # __call__ model() in this case: __call__ internally calls forward()
            # [batch_size, num_classes]

            loss = self.loss_fn(logits, batch[1].view(-1).to(self.device)) # Compute loss: Cross entropy loss

            loss_history.append(loss.item())

            

            running_loss += (loss_history[-1] - running_loss) / (i + 1) # Compute rolling average

            if self.log_every_n and i % self.log_every_n == 0:
                print("Running loss: ", running_loss)

            running_loss_history.append(running_loss)

            loss.backward() # Perform backprop, which will compute dL/dw

            nn.utils.clip_grad_norm_(self.model.parameters(), 3.0) # We clip gradient's norm to 3

            self.optimizer.step() # Update step: w = w - eta * dL / dW : eta = 1e-2 (0.01), gradient = 5e30; update value of 5e28

        print("Epoch completed!")
        print("Epoch Loss: ", running_loss)
        print("Epoch Perplexity: ", math.exp(running_loss))

        # The history information can allow us to draw a loss plot
        return loss_history, running_loss_history

    def evaluate(self, loader, labels):
        """
        Evaluate the model on a validation set
        """

        self.model.eval() # Run model in eval mode (disables dropout layer)

        batch_wise_true_labels = []
        batch_wise_predictions = []

        loss_history = []
        running_loss = 0.
        running_loss_history = []

        with torch.no_grad(): # Disable gradient computation - required only during training
            for i, batch in tqdm(enumerate(loader)):
                # batch[0] shape: (batch_size, input_size)

                logits = self.model(batch[0].to(self.device)) # Run forward pass (except we don't store gradients)
                # logits shape: (batch_size, num_classes)
                
                loss = self.loss_fn(logits, batch[1].view(-1).to(self.device)) # Compute loss
                # No backprop is done during validation
                
                # Instead of using CrossEntropyLoss, you use BCEWithLogitsLoss
                # BCEWithLogitsLoss - independently calculates loss for each class
                

                loss_history.append(loss.item())

                running_loss += (loss_history[-1] - running_loss) / (i + 1) # Compute rolling average
                
                running_loss_history.append(running_loss)

                # logits : [batch_size, num_classes] and each of the values in logits can be anything (-infinity, +infity)
                # Converts the raw outputs into probabilities for each class using softmax
                probs = F.softmax(logits, dim=-1) 
                # probs shape: (batch_size, num_classes)
                # -1 dimension picks the last dimension in the shape of the tensor, in this case 'num_classes'
                

                # softmax vector: [[0.1, 0.2, 0.6, 0.1, 0.0], [0.9, 0.01, 0.01, 0.01, 0.07]]
                # output tensor: [2, 0]
                predictions = torch.argmax(probs, dim=-1) # Output predictions; Argmax picks the index with the highest probability among all the classes (choosing our most probable class)
                # predictions shape: (batch_size)

                batch_wise_true_labels.append(batch[1].tolist())
                batch_wise_predictions.append(predictions.tolist())
        
        # flatten the list of predictions using itertools
        all_true_labels = list(chain.from_iterable(batch_wise_true_labels))
        all_predictions = list(chain.from_iterable(batch_wise_predictions))

        # Now we can generate a classification report
        print("Classification report after epoch:")
        print(classification_report(all_true_labels, all_predictions, target_names=labels))

        return loss_history, running_loss_history

    def get_model_dict(self):
        return self.model.state_dict()

    def run_training(self, train_loader, valid_loader, labels, n_epochs=10):
        # Useful for us to review what experiment we're running
        # Normally, you'd want to save this to a file
        self._print_summary()

        train_losses = []
        train_running_losses = []

        valid_losses = []
        valid_running_losses = []

        for i in range(n_epochs):
            loss_history, running_loss_history = self.train(train_loader)
            valid_loss_history, valid_running_loss_history = self.evaluate(valid_loader, labels)

            train_losses.append(loss_history)
            train_running_losses.append(running_loss_history)

            valid_losses.append(valid_loss_history)
            valid_running_losses.append(valid_running_loss_history)

        # Training done, let's look at the loss curves
        all_train_losses = list(chain.from_iterable(train_losses))
        all_train_running_losses = list(chain.from_iterable(train_running_losses))

        all_valid_losses = list(chain.from_iterable(valid_losses))
        all_valid_running_losses = list(chain.from_iterable(valid_running_losses))

        train_epoch_idx = range(len(all_train_losses))
        valid_epoch_idx = range(len(all_valid_losses))
        # sns.lineplot(epoch_idx, all_losses)
        sns.lineplot(train_epoch_idx, all_train_running_losses)
        sns.lineplot(valid_epoch_idx, all_valid_running_losses)
        plt.show()

In [65]:
# prepare data
train_data_loader = DataLoader(dataset, batch_size=5)
val_data_loader = DataLoader(dataset, batch_size=5)


In [66]:
# Define our experimental setup; we use the same parameters for both MLP models
input_size = dataset.input_text_len
hidden_size = 100 # An arbitrary hyperparameter we define
output_size = dataset.n_labels
print(input_size)
print(output_size)

# Our experimental hyperparameters
LEARNING_RATE = 1e-2

loss_fn = nn.BCEWithLogitsLoss()

50
19


In [67]:
mlp = MultiLayerPerceptron(input_size, hidden_size, output_size)
optimizer = optim.SGD(mlp.parameters(), lr=LEARNING_RATE, momentum=0.9)


mlp_trainer = MultiClassTrainer(mlp, optimizer, loss_fn)
mlp_trainer.run_training(train_data_loader, val_data_loader, dataset.labels, n_epochs=2)

MultiLayerPerceptron(
  (input_layer): Linear(in_features=50, out_features=100, bias=True)
  (hidden_layer): Linear(in_features=100, out_features=19, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
BCEWithLogitsLoss()


0it [00:00, ?it/s]


RuntimeError: ignored